In [1]:
import cv2
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
c = 0
for files in os.listdir("./train"):
    c += 1
print(c)

25000


In [3]:
txt = "cat.1.jpg"

x = txt.split(".")

print(x)

['cat', '1', 'jpg']


In [4]:
data = []
for files in os.listdir('./train'):
    image = cv2.resize(cv2.imread(os.path.join('./train', files)), (100, 100))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    label = files.split('.')[0]
    if label == 'cat':
        data.append([gray, np.array([0, 1])])
    elif label == 'dog':
        data.append([gray, np.array([1, 0])])

In [5]:
len(data)

25000

In [6]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 1), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 98, 98, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 47, 47, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 32)        1

In [7]:
train , test = train_test_split(data, test_size = 0.1, shuffle = True)

In [8]:
print(len(train))
print(len(test))

22500
2500


In [9]:
test[5][1]

array([1, 0])

In [10]:
X = np.array([i[0] for i in train]).reshape(-1, 100, 100, 1)
Y = np.array([i[1] for i in train])

x_valid = np.array([i[0] for i in test]).reshape(-1, 100, 100, 1)
y_valid = np.array([i[1] for i in test])

In [11]:
type(X)

numpy.ndarray

In [12]:
X = X.astype('float32')/255
x_valid = x_valid.astype('float32')/255

In [13]:
print(X.shape)
print(Y.shape)
print(x_valid.shape)
print(y_valid.shape)

(22500, 100, 100, 1)
(22500, 2)
(2500, 100, 100, 1)
(2500, 2)


In [14]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss= 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])

In [15]:
model.fit(X, Y, batch_size= 10, epochs= 3, verbose= 1, validation_data=(x_valid,y_valid))

Epoch 1/3
2250/2250 [==============================] - 1166s 509ms/step - loss: 0.6604 - accuracy: 0.5915 - val_loss: 0.6248 - val_accuracy: 0.6488
Epoch 2/3
2250/2250 [==============================] - 963s 428ms/step - loss: 0.5874 - accuracy: 0.6909 - val_loss: 0.5493 - val_accuracy: 0.7180
Epoch 3/3
2250/2250 [==============================] - 1061s 471ms/step - loss: 0.5149 - accuracy: 0.7469 - val_loss: 0.4929 - val_accuracy: 0.7588
